In [1]:
pip install transformers datasets pandas scikit-learn


In [3]:
import pandas as pd
from datasets import Dataset

# Load cleaned CSV (with Feedback as Yes/No)
df = pd.read_csv("reward_training_data_55_clean.csv")

# Keep only rows with clear feedback
df = df[df["Feedback"].isin(["Yes", "No"])]

# Map to binary labels
df["label"] = df["Feedback"].map({"Yes": 1, "No": 0})

# Rename for compatibility
df = df[["Answer", "label"]].rename(columns={"Answer": "text"})

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

tokenized = dataset.map(tokenize, batched=True)
split = tokenized.train_test_split(test_size=0.2)
train_ds = split["train"]
test_ds = split["test"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

In [5]:
pip install --upgrade transformers


In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [8]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./rm_model1",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir="./rm_logs1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
)

trainer.train()
trainer.save_model("./rm_model1")
print("✅ Model trained and saved.")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


✅ Model trained and saved.


In [9]:
tokenizer.save_pretrained("./rm_model2")


('./rm_model2/tokenizer_config.json',
 './rm_model2/special_tokens_map.json',
 './rm_model2/vocab.txt',
 './rm_model2/added_tokens.json',
 './rm_model2/tokenizer.json')

In [10]:
model.save_pretrained("./rm_model", safe_serialization=False)  # saves model as .bin
tokenizer.save_pretrained("./rm_model")


('./rm_model/tokenizer_config.json',
 './rm_model/special_tokens_map.json',
 './rm_model/vocab.txt',
 './rm_model/added_tokens.json',
 './rm_model/tokenizer.json')

In [11]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model = AutoModelForSequenceClassification.from_pretrained("./rm_model")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Score from 0 to 1 (higher is better)
def score_answer(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        score = torch.softmax(logits, dim=1)[0][1].item()
    return score

#  Example scoring
print(score_answer("Assignment 2 is due on March 14th. Submit on Moodle."))  # High expected
print(score_answer("I don't know. Try asking someone else."))              # Low expected


0.9200415015220642
0.41946011781692505
